In [2]:
from textblob import TextBlob
import arff
from io import StringIO
from sklearn.model_selection import train_test_split
from textblob.classifiers import NaiveBayesClassifier
import nltk
import pandas as pd

In [13]:
data = arff.load(open('OffComBR3.arff'))

In [14]:
df = pd.DataFrame(data['data'])
df.columns = ['hate', 'sentence']

In [5]:
def transform_string_boolen_to_int(x):
    if x == 'yes':
        retorno = 1
    elif x == 'no':
        retorno = 0
    else:
        retorno = x
    return retorno
df['hate'] = df['hate'].apply(transform_string_boolen_to_int)

In [15]:
X = df.drop(['hate'], axis=1)
y = df['sentence']
#X_train, X_test, y_train, y_test = train_test_split(X.values,y,test_size=0.33)
train = df.iloc[:723, :]
test = df.iloc[723:, :]

In [10]:
# print('Data sizes:'
#         + '\nX_train: ' + str(len(X_train))
#         + '\nX_test: ' + str(len(X_test))
#         + '\ny_train: ' + str(len(y_train))
#         + '\ny_test: ' + str(len(y_test))
#      )

In [69]:
# this method return the traing data in the TextBlob recommended format
def  build_txblob_array (data):
    txb_array = []
    for d in data.iterrows():
        new_data = (d[1][1], d[1][0])
        txb_array.append(new_data)
    return(txb_array)

In [71]:
# building arrays to train and test the textBlob's NaiveBayesClassifier
train_array = build_txblob_array(train)
test_array = build_txblob_array(test)

In [72]:
# creating classifier
cl = NaiveBayesClassifier(train_array)

In [75]:
# testing classifier
cl.accuracy(test_array)

0.6129032258064516

In [89]:
cl.classify('Mais um pobre metido a besta')

'no'